In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
CSV_PATH = r"C:\Users\asila\Downloads\strsearch_takehome\data\Blue Ridge GA - Market Eval - FINAL - Base_Table.csv"

In [3]:
df = pd.read_csv(CSV_PATH, low_memory=False)

In [4]:
print("Shape:", df.shape)
display(df.head(3))

Shape: (906, 101)


,Property ID,TITLE,Property Manager/ Host ID,BEDROOMS,ACCOMMODATES,Airbnb Host URL,Airbnb Listing URL,BATHROOMS,CITY_NAME,HAS_AIRCON,...,latitude,longitude,is_super_host,is_guest_favorite,Has_Outdoor_Furniture,Has_Waterfront,Has_Lake_Access,Has_Beach_Access,Has_Outdoor_Dining_Area,error_reason
0,abnb_843427409328707195,Lake Front Luxury Pool Hot tub Movie Theater Gym,504577995,10,16,https://www.airbnb.com/users/show/504577995,https://www.airbnb.com/rooms/843427409328707195,9.0,Morganton,True,...,34.854862,-84.267015,True,True,True,True,False,False,True,NaN
1,abnb_1072358047706308102,"Views, Pool, Sport Court, Gym, Sauna, Petting Zoo",190951764,7,16,https://www.airbnb.com/users/show/190951764,https://www.airbnb.com/rooms/1072358047706308102,5.5,Blue Ridge,True,...,34.920260,-84.386820,True,True,True,False,False,False,True,NaN
2,abnb_843427434784984731,Luxury Riverfront Pool Theater Games Hot Tub,504577995,9,16,https://www.airbnb.com/users/show/504577995,https://www.airbnb.com/rooms/843427434784984731,7.5,Mineral Bluff,True,...,34.905413,-84.310215,True,True,True,True,False,False,True,NaN


In [6]:
for c in df.columns[:101]:
    print("-", c)

- Property ID
- TITLE
- Property Manager/ Host ID
- BEDROOMS
- ACCOMMODATES
- Airbnb Host URL
- Airbnb Listing URL
- BATHROOMS
- CITY_NAME
- HAS_AIRCON
- HAS_GYM
- HAS_HOTTUB
- HAS_KITCHEN
- HAS_PARKING
- HAS_PETS_ALLOWED
- HAS_POOL
- INSTANT_BOOK
- LATITUDE
- LONGITUDE
- PRICE_TIER
- STATE_NAME
- SUPERHOST
- Vrbo Listing URL
- ZIPCODE
- MINIMUM_STAY
- Available Nights
- Revenue
- Revenue Potential
- ADR
- Occupancy
- Cleaning Fee
- Property Reviews
- Property Rating
- Listing Name
- Data Quality Category
- Quality Rating Reason
- Bedrooms
- total_reviews
- total_months
- missing_months
- avg_reviews_per_month
- high_season_reviews
- high_season
- High Season Insights
- numberOfGuests
- url
- name
- Location
- stars
- description
- personCapacity
- beds
- number_of_beds
- baths
- title
- propertyType
- roomType
- reviewsCount
- amenities
- SYSTEM_GYM
- SYSTEM_POOL_TABLE
- SYSTEM_ARCADE_MACHINE
- SYSTEM_MOVIE
- SYSTEM_BOWLING
- SYSTEM_CHESS
- SYSTEM_GOLF
- SYSTEM_CRIB
- SYSTEM_PACK_N_PL

In [8]:
dtypes = df.dtypes.astype(str).rename("dtype").to_frame()
missing = df.isna().mean().rename("missing_frac").to_frame()
unique = df.nunique(dropna=True).rename("n_unique").to_frame()
summary = dtypes.join(missing).join(unique).sort_values(["missing_frac", "n_unique"], ascending=[False, True])

display(summary.head(101))

,dtype,missing_frac,n_unique
Location,float64,1.000000,0
error_reason,object,0.963576,1
Vrbo Listing URL,object,0.108168,808
review_high_season_label,object,0.041943,261
high_season,float64,0.036424,1
...,...,...,...
TITLE,object,0.000000,905
Property ID,object,0.000000,906
Airbnb Listing URL,object,0.000000,906
Revenue Potential,float64,0.000000,906


In [9]:
display(summary["dtype"].value_counts())

dtype
object     49
float64    36
bool        8
int64       8
Name: count, dtype: int64

In [10]:
def is_boolish_series(s: pd.Series) -> bool:
    # drop missing
    v = s.dropna()
    if v.empty:
        return False
    # take a small sample
    v = v.sample(min(len(v), 1000), random_state=0)

    # normalize strings
    vals = set()
    for x in v:
        if isinstance(x, str):
            vals.add(x.strip().lower())
        else:
            vals.add(x)

    boolish_sets = [
        {"true", "false"},
        {"t", "f"},
        {"yes", "no"},
        {"y", "n"},
        {0, 1},
        {0.0, 1.0},
    ]
    # also accept mix like {"true", "false", 1, 0} by checking subset of known boolish tokens
    known_tokens = {"true","false","t","f","yes","no","y","n",0,1,0.0,1.0}

    if vals.issubset(known_tokens):
        # Need at least two values to be meaningful
        return len(vals) >= 2
    return False

boolish_cols = [c for c in df.columns if is_boolish_series(df[c])]
print(f"\n✅ Detected {len(boolish_cols)} boolean-ish columns (likely amenities/flags). Showing first 40:")
print(boolish_cols[:40])


✅ Detected 32 boolean-ish columns (likely amenities/flags). Showing first 40:
['HAS_AIRCON', 'HAS_GYM', 'HAS_HOTTUB', 'HAS_KITCHEN', 'HAS_PARKING', 'HAS_PETS_ALLOWED', 'HAS_POOL', 'INSTANT_BOOK', 'SUPERHOST', 'SYSTEM_GYM', 'SYSTEM_POOL_TABLE', 'SYSTEM_ARCADE_MACHINE', 'SYSTEM_MOVIE', 'SYSTEM_BOWLING', 'SYSTEM_CHESS', 'SYSTEM_GOLF', 'SYSTEM_CRIB', 'SYSTEM_PACK_N_PLAY', 'SYSTEM_PLAY_SLIDE', 'SYSTEM_FIREPIT', 'SYSTEM_GRILL', 'SYSTEM_POOL', 'SYSTEM_JACUZZI', 'SYSTEM_VIEW_OCEAN', 'SYSTEM_VIEW_MOUNTAIN', 'is_super_host', 'is_guest_favorite', 'Has_Outdoor_Furniture', 'Has_Waterfront', 'Has_Lake_Access', 'Has_Beach_Access', 'Has_Outdoor_Dining_Area']


In [11]:
def normalize_colname(c: str) -> str:
    c = c.strip().lower()
    c = re.sub(r"[^a-z0-9]+", "", c)  # remove spaces/underscores/punct
    return c

norm_map = {}
for c in df.columns:
    norm = normalize_colname(c)
    norm_map.setdefault(norm, []).append(c)

dupe_like = {k:v for k,v in norm_map.items() if len(v) > 1}
print(f"\n⚠️ Found {len(dupe_like)} groups of duplicate-ish column names:")
# show a few
for i, (k, cols) in enumerate(list(dupe_like.items())[:20], start=1):
    print(f"{i:02d}. {cols}")

# --- Quick check of key candidate columns (you can edit this list) ---
key_candidates = [
    "Property ID", "TITLE", "title", "Listing Name", "name",
    "Bedrooms", "BEDROOMS", "Bathrooms", "ACCOMMODATES", "personCapacity",
    "Revenue", "Potential Revenue", "ADR", "Occupancy", "Cleaning Fee",
    "latitude", "LATITUDE", "longitude", "LONGITUDE", "Zipcode", "zipcode",
    "Airbnb Listing URL", "Vrbo Listing URL"
]

present = [c for c in key_candidates if c in df.columns]
print("\n✅ Key candidate columns present:")
print(present)


⚠️ Found 4 groups of duplicate-ish column names:
01. ['TITLE', 'title']
02. ['BEDROOMS', 'Bedrooms']
03. ['LATITUDE', 'latitude']
04. ['LONGITUDE', 'longitude']

✅ Key candidate columns present:
['Property ID', 'TITLE', 'title', 'Listing Name', 'name', 'Bedrooms', 'BEDROOMS', 'ACCOMMODATES', 'personCapacity', 'Revenue', 'ADR', 'Occupancy', 'Cleaning Fee', 'latitude', 'LATITUDE', 'longitude', 'LONGITUDE', 'Airbnb Listing URL', 'Vrbo Listing URL']


In [12]:
cols_to_show = present[:10]
if cols_to_show:
    display(df[cols_to_show].head(5))

,Property ID,TITLE,title,Listing Name,name,Bedrooms,BEDROOMS,ACCOMMODATES,personCapacity,Revenue
0,abnb_843427409328707195,Lake Front Luxury Pool Hot tub Movie Theater Gym,Lake Front Luxury Pool Hot tub Movie Theater Gym,Lake Front Luxury Pool Hot tub Movie Theater Gym,Lake Front Luxury Pool Hot tub Movie Theater Gym,10.0,10,16,16.0,496859
1,abnb_1072358047706308102,"Views, Pool, Sport Court, Gym, Sauna, Petting Zoo","Views, Pool, Sport Court, Gym, Sauna, Petting Zoo","Views, Pool, Sport Court, Gym, Sauna, Petting Zoo","Views, Pool, Sport Court, Gym, Sauna, Petting Zoo",7.0,7,16,16.0,326334
2,abnb_843427434784984731,Luxury Riverfront Pool Theater Games Hot Tub,Luxury Riverfront Pool Theater Games Hot Tub,Luxury Riverfront Pool Theater Games Hot Tub,Luxury Riverfront Pool Theater Games Hot Tub,9.0,9,16,16.0,337755
3,abnb_47115055,"Toccoa Lodge | Toccoa River Frontage | 10,000 ...","Toccoa Lodge | Toccoa River Frontage | 10,000 ...","Toccoa Lodge | Toccoa River Frontage | 10,000 ...","Toccoa Lodge | Toccoa River Frontage | 10,000 ...",8.0,8,16,16.0,257005
4,abnb_17864330,Luxe Mountain Resort - Heated Pool & 12ft Hot Tub,Luxe Mountain Resort - Heated Pool & 12ft Hot Tub,Luxe Mountain Resort - Heated Pool & 12ft Hot Tub,Luxe Mountain Resort - Heated Pool & 12ft Hot Tub,6.0,6,16,16.0,266719
